Tests integration of all three models to see if it works.

In [1]:
import os
import time

import torch 
import numpy as np
import face_alignment

import cheapfake.contrib.dataset as dataset
from cheapfake.contrib.models import CheapFake
import cheapfake.contrib.transforms as transforms
import cheapfake.contrib.video_processor as video_processor

In [22]:
random_seed = 41
root_path = "/mnt/fs03/shared/datasets/dfdc_train_all"

dfdataset = dataset.DeepFakeDataset(
    root_path=root_path, frame_transform=transforms.BatchRescale(4), sequential_audio=True
)
frames, audio, audio_stft = dfdataset.__getitem__(0)
frames = frames[:75]
print(frames.shape)
print(audio.shape)
print(audio_stft.shape)

torch.Size([75, 3, 270, 480])
torch.Size([48000])
torch.Size([94, 1025])


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
model = CheapFake(device=device)

In [5]:
def _forward_fan(x):
    fan_output = model.face_alignment_model.get_landmarks_from_batch(x)
    predictions = np.asarray(fan_output).squeeze(axis=1)
    
    # 45 -> 55 is the range of the two lip areas horizontally. 
    # 52 -> 58 is the range of the two lip areas vertically.
    # These are the coordinates of the bounding box that will be used to crop the lip areas.
    lip_range_coords = [predictions[:, 45,:], predictions[:, 52,:], predictions[:, 55,:], predictions[:, 58,:]]
    
    # Crop the images once the coordinates are obtained.
    """
    tolerance = 30
    x = np.einsum("ijkl->iklj", x.cpu().numpy())
    cropped_lips = torch.empty(x.shape)
    for k, frame in enumerate(x):
        cropped_lips[k] = torch.from_numpy(frame[
            int(lip_range_coords[0][k][0]):int(lip_range_coords[2][k][0]), 
            int(lip_range_coords[1][k][1]):int(lip_range_coords[3][k][1])
        ]).cuda()
    """
    
    return predictions, lip_range_coords


In [6]:
start_time = time.time()
predictions, lip_range_coords = _forward_fan(frames.float().cuda())
end_time = time.time()

print("Entire operation took {} seconds".format(end_time - start_time))

Entire operation took 45.01742911338806 seconds


In [27]:
print(int(lip_range_coords[0][0][0]))
print(int(lip_range_coords[2][k][0]))

240
235


In [23]:
cropped_lips = torch.empty((frames.shape[0], frames.shape[2], frames.shape[3], frames.shape[1]))
frames = torch.from_numpy(np.einsum("ijkl->iklj", frames))
for k, frame in enumerate(frames):
    cropped_lips[k] = frame[
        int(lip_range_coords[0][k][0]):int(lip_range_coords[2][k][0]),
        int(lip_range_coords[1][k][1]):int(lip_range_coords[3][k][1]),
    ]

print(cropped_lips.shape)

RuntimeError: The expanded size of the tensor (480) must match the existing size (6) at non-singleton dimension 1.  Target sizes: [270, 480, 3].  Tensor sizes: [0, 6, 3]